In [ ]:
# import required packages
from __future__ import print_function, division
#import cv2
import dlib
import time
from skimage import io
import os
import shutil
import torch
import pandas as pd
import torchvision
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.stats.api as sms
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from os import listdir
from os.path import isfile, join
from collections import namedtuple
from skimage.transform import rescale
from skimage.transform import resize
import sys
import glob
import PIL
import os
import shutil
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
import math
import PIL
import imgaug as ia
from imgaug import augmenters as iaa
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from pathlib import Path
from collections import Counter
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
print(os.getcwd())
#np.random.seed(198467)
torch.cuda.empty_cache()

In [2]:
frame = pd.read_csv('ExpW_OpenFace_result_with_expression_gender_race_age.csv')
frame = frame[['ImageName', 'Expression', 'gender_preds']]
frame.gender_preds = frame.gender_preds.apply(lambda x: 1 if x == "Male" else 0)
frame = frame[['ImageName', 'Expression', 'gender_preds']]
frame.Expression = frame.Expression.apply(lambda x: 1 if x == 3 else 0)
frame.head()

,ImageName,Expression,gender_preds
0,angry_actor_104_0.jpg,0,1
1,angry_actor_109_0.jpg,0,1
2,angry_actor_120_1.jpg,0,1
3,angry_actor_13_0.jpg,1,0
4,angry_actor_132_0.jpg,0,0


In [3]:
frame.shape

(88600, 3)

In [4]:
frame.Expression.value_counts()

0    59707
1    28893
Name: Expression, dtype: int64

In [5]:
frame.gender_preds.value_counts()

1    61108
0    27492
Name: gender_preds, dtype: int64

In [6]:
frame['ImageName'] = frame['ImageName'].apply(lambda x: '/sdata/ExpW_cropped_by_their_coordinates/' + x) # change ImageName to full path
frame_copy = frame.copy()

In [7]:
result_rows_list = []

In [8]:
for k in range(1,6):

    # Split into train/validation/test sets
    frame = frame_copy.sample(n = 20000, random_state = k).reset_index(drop=True) # shuffle data frame
    n_images = len(frame)
    n_train = int(0.8 * n_images)
    n_val = int((n_images - n_train) / 2)
    n_test = n_images - n_train - n_val

    train_frame = frame[0 : n_train].reset_index(drop=True)
    val_frame = frame[n_train : n_train + n_val].reset_index(drop=True)
    test_frame = frame[n_train + n_val : ].reset_index(drop=True)

    print("{} train faces, {} validation faces, {} test faces".format(len(train_frame), len(val_frame), len(test_frame)))

    # Data loaders and transforms for training

    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
    # image.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image.
    class ImgAugTransform:

        def __init__(self):
            self.aug = iaa.Sequential(
            [
                #
                # Apply the following augmenters to most images.
                #
                iaa.Fliplr(0.5), # horizontally flip 50% of all images
                #iaa.Flipud(0.2), # vertically flip 20% of all images

                # crop some of the images by 0-10% of their height/width
                sometimes(iaa.Crop(percent=(0, 0.05))),

                # Apply affine transformations to some of the images
                # - scale to 80-120% of image height/width (each axis independently)
                # - translate by -20 to +20 relative to height/width (per axis)
                # - rotate by -45 to +45 degrees
                # - shear by -16 to +16 degrees
                # - order: use nearest neighbour or bilinear interpolation (fast)
                # - mode: use any available mode to fill newly created pixels
                #         see API or scikit-image for which modes are available
                # - cval: if the mode is constant, then use a random brightness
                #         for the newly created pixels (e.g. sometimes black,
                #         sometimes white)
                iaa.Affine(
                    scale={"x": (1, 1.1), "y": (1, 1.1)}, # scale images to 80-120% of their size, individually per axis
                    translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -10 to +10 percent (per axis)
                    rotate=(-15, 15), # rotate by -15 to +15 degrees
                    shear=(-8, 8), # shear by -8 to +8 degrees
                    order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                    #cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                    mode=['edge'] # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                ),

                #
                # Execute 0 to 5 of the following (less important) augmenters per
                # image. Don't execute all of them, as that would often be way too
                # strong.
                #
                iaa.SomeOf((0, 5),
                    [
                        # Convert some images into their superpixel representation,
                        # sample between 20 and 200 superpixels per image, but do
                        # not replace all superpixels with their average, only
                        # some of them (p_replace).
                        sometimes(
                            iaa.Superpixels(
                                p_replace=(0, 0.1),
                                n_segments=(50, 200)
                            )
                        ),

                        # Blur each image with varying strength using
                        # gaussian blur (sigma between 0 and 3.0),
                        # average/uniform blur (kernel size between 2x2 and 7x7)
                        # median blur (kernel size between 3x3 and 11x11).
                        iaa.OneOf([
                            iaa.GaussianBlur((0, 3.0)),
                            iaa.AverageBlur(k=(2, 7)),
                            iaa.MedianBlur(k=(3, 11)),
                        ]),

                        # Sharpen each image, overlay the result with the original
                        # image using an alpha between 0 (no sharpening) and 1
                        # (full sharpening effect).
                        iaa.Sharpen(alpha=(0, 0.3), lightness=(0.75, 1.5)),

                        # Same as sharpen, but for an embossing effect.
                        iaa.Emboss(alpha=(0, 0.3), strength=(0, 2)),

                        # Search in some images either for all edges or for
                        # directed edges. These edges are then marked in a black
                        # and white image and overlayed with the original image
                        # using an alpha of 0 to 0.7.
                        sometimes(iaa.OneOf([
                            iaa.EdgeDetect(alpha=(0, 0.3)),
                            iaa.DirectedEdgeDetect(
                                alpha=(0, 0.3), direction=(0.0, 1.0)
                            ),
                        ])),

                        # Add gaussian noise to some images.
                        # In 50% of these cases, the noise is randomly sampled per
                        # channel and pixel.
                        # In the other 50% of all cases it is sampled once per
                        # pixel (i.e. brightness change).
                        iaa.AdditiveGaussianNoise(
                            loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                        ),

                        # Either drop randomly 1 to 10% of all pixels (i.e. set
                        # them to black) or drop them on an image with 2-5% percent
                        # of the original size, leading to large dropped
                        # rectangles.
                        iaa.OneOf([
                            iaa.Dropout((0.01, 0.02), per_channel=0.5),
                            #iaa.CoarseDropout(
                            #    (0.03, 0.15), size_percent=(0.02, 0.05),
                            #    per_channel=0.2
                            #),
                        ]),

                        # Invert each image's chanell with 5% probability.
                        # This sets each pixel value v to 255-v.
                        #iaa.Invert(0.05, per_channel=True), # invert color channels

                        # Add a value of -10 to 10 to each pixel.
                        iaa.Add((-15, 15), per_channel=0.5),

                        # Change brightness of images (50-150% of original value).
                        iaa.Multiply((0.75, 1.25), per_channel=0.5),

                        # Improve or worsen the contrast of images.
                        iaa.ContrastNormalization((0.75, 1.75), per_channel=0.5),

                        # Convert each image to grayscale and then overlay the
                        # result with the original with random alpha. I.e. remove
                        # colors with varying strengths.
                        iaa.Grayscale(alpha=(0.0, 1.0)),

                        # In some images move pixels locally around (with random
                        # strengths).
                        #sometimes(
                        #    iaa.ElasticTransformation(alpha=(0.1, 0.2), sigma=0.25)
                        #),

                        # In some images distort local areas with varying strength.
                        sometimes(iaa.PiecewiseAffine(scale=(0.005, 0.01)))
                    ],
                    # do all of the above augmentations in random order
                    random_order=True
                )
            ],
            # do all of the above augmentations in random order
            random_order=True
        )

        def __call__(self, img):

            img = np.array(img)
            return self.aug.augment_image(img)

    class ImageDataset(Dataset):

        def __init__(self, data_frame, transform=None):

            self.data_frame = data_frame
            self.transform = transform

        def __len__(self):

            return len(self.data_frame)

        def __getitem__(self, idx):
            #idx is index from dataset
            #This is a mapping from your data_frame to the output of the mode
            img_name = self.data_frame.loc[idx, 'ImageName']

            expression = self.data_frame.loc[idx, 'Expression']
            gender = self.data_frame.loc[idx, 'gender_preds']

            # read image as ndarray, H*W*C
            image = dlib.load_rgb_image(img_name)       
            image = cv2.resize(image, (224,224)) # resize the image to 224x224 for the ResNet Model

            if self.transform:
                image = self.transform(image)

            # transform label to torch tensor
            # This sets the order of the label
            return (image, torch.from_numpy(np.asarray(expression, dtype=np.float32)), 
                    torch.from_numpy(np.asarray(gender, dtype=np.float32)))


    transform_train_data = transforms.Compose([
        ImgAugTransform(),
        lambda x: PIL.Image.fromarray(x),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    transformed_train_dataset = ImageDataset(data_frame=train_frame,
                                               transform=transform_train_data
                                               )

    train_dataloader = DataLoader(transformed_train_dataset, batch_size=64,
                            shuffle=False, num_workers=8)


    transform_val_data = transforms.Compose(([transforms.ToPILImage(), 
                                          transforms.ToTensor(), 
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                              ]))

    transformed_val_dataset = ImageDataset(data_frame=val_frame,
                                               transform=transform_val_data
                                               )

    val_dataloader = DataLoader(transformed_val_dataset, batch_size=64,
                            shuffle=False, num_workers=8)

    transform_test_data = transforms.Compose(([transforms.ToPILImage(), 
                                          transforms.ToTensor(), 
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                              ]))

    transformed_test_dataset = ImageDataset(data_frame=test_frame,
                                               transform=transform_test_data
                                               )

    test_dataloader = DataLoader(transformed_test_dataset, batch_size=64,
                            shuffle=False, num_workers=8)

    # Training

    torch.cuda.is_available()

    dataloaders = {'train': train_dataloader, 'test': test_dataloader}
    dataset_sizes = {'train': len(transformed_train_dataset), 'test': len(transformed_test_dataset)}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # utils.py

    import json
    import pickle
    from sklearn.metrics import average_precision_score

    def save_pkl(pkl_data, save_path):
        with open(save_path, 'wb') as f:
            pickle.dump(pkl_data, f)

    def load_pkl(load_path):
        with open(load_path, 'rb') as f:
            pkl_data = pickle.load(f)
        return pkl_data

    def save_json(json_data, save_path):
        with open(save_path, 'w') as f:
            json.dump(json_data, f)

    def load_json(load_path):
        with open(load_path, 'r') as f:
            json_data = json.load(f)
        return json_data

    def save_state_dict(state_dict, save_path):
        torch.save(state_dict, save_path)

    def creat_folder(path):
        if not os.path.exists(path):
            os.makedirs(path)

    def set_random_seed(seed_number):
        torch.manual_seed(seed_number)
        np.random.seed(seed_number)

    def write_info(filename, info):
        with open(filename, 'w') as f:
            f.write(info)

    def compute_weighted_AP(target, predict_prob, class_weight_list):
        per_class_AP = []
        for i in range(target.shape[1] - 1):
            class_weight = target[:, i]*class_weight_list[i] \
                           + (1-target[:, i])*np.ones(class_weight_list[i].shape)
            per_class_AP.append(average_precision_score(target[:, i], predict_prob[:], 
                                    sample_weight=class_weight))

        return per_class_AP

    def compute_mAP(per_class_AP):
        return np.mean([per_class_AP[idx] for idx in [0]])

    def compute_class_weight(target):
        domain_label = target[:, -1]
        per_class_weight = []

        for i in range(target.shape[1]-1):
            class_label = target[:, i]
            cp = class_label.sum() # class is positive
            cn = target.shape[0] - cp # class is negative
            cn_dn = ((class_label + domain_label)==0).sum() # class is negative, domain is negative
            cn_dp = ((class_label - domain_label)==-1).sum()
            cp_dn = ((class_label - domain_label)==1).sum()
            cp_dp = ((class_label + domain_label)==2).sum()

            per_class_weight.append(
                (class_label*cp + (1-class_label)*cn) / 
                    (2*(
                        (1-class_label)*(1-domain_label)*cn_dn
                        + (1-class_label)*domain_label*cn_dp
                        + class_label*(1-domain_label)*cp_dn
                        + class_label*domain_label*cp_dp
                       )
                    )
            )
        return per_class_weight

    # celeba_uniconf_adv.py
    class ResNet50_base(nn.Module):   
        """ResNet50 but without the final fc layer"""

        def __init__(self, pretrained, hidden_size=2048, dropout=0.5):
            super().__init__()
            self.resnet = torchvision.models.resnet50(pretrained=pretrained)                
            self.resnet.fc = nn.Linear(2048, hidden_size)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(dropout)        

        def require_all_grads(self):
            for param in self.parameters():
                param.requires_grad = True

        def forward(self, x):
            features = self.resnet(x)
            features = self.dropout(self.relu(features))

            return features

    class CelebaUniConfAdv():
        def __init__(self):
            self.training_ratio = 3
            self.alpha = 2
            self.epoch = 0
            self.best_dev_mAP = 0.
            self.train_loader = train_dataloader    
            self.dev_loader = val_dataloader        
            self.test_loader = test_dataloader

            self.dev_target = val_frame.iloc[:,[1,2]].to_numpy()
            self.dev_class_weight = compute_class_weight(self.dev_target)
            self.test_target = test_frame.iloc[:,[1,2]].to_numpy()
            self.test_class_weight = compute_class_weight(self.test_target)


            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

            self.base_network = ResNet50_base(pretrained=True).to(self.device)


            num_female = len(train_frame[train_frame.gender_preds == 0])
            num_male = len(train_frame[train_frame.gender_preds == 1])
            weight_gender = torch.tensor([num_male/num_female, num_male/num_male]).to(self.device)

            self.criterion = nn.CrossEntropyLoss(weight = weight_gender)
            # Two fc layers on top of the base network, one for target classification,
            # one for domain classification
            self.class_network = nn.Linear(2048, 1).to(self.device)
            self.domain_network = nn.Linear(2048, 2).to(self.device)



            self.base_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.base_network.parameters()), 
                                      lr=1e-4)
            self.class_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.class_network.parameters()), 
                                      lr=1e-4)
            self.domain_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.domain_network.parameters()), 
                                      lr=1e-4)

        def _criterion(self, output, expression):
            return F.binary_cross_entropy_with_logits(torch.squeeze(output), expression.float())
            #return self.criterion(torch.squeeze(output), expression.long())

        def _domain_criterion(self, output, gender):

            return self.criterion(torch.squeeze(output), gender.long())

        def state_dict(self):
            state_dict = {
                'base_network': self.base_network.state_dict(),
                'class_network': self.class_network.state_dict(),
                'domain_network': self.domain_network.state_dict(),
                'base_optimizer': self.base_optimizer.state_dict(),
                'class_optimizer': self.class_optimizer.state_dict(),
                'domain_optimizer': self.domain_optimizer.state_dict(),
                'epoch': self.epoch
            }
            return state_dict

        def load_state_dict(self, state_dict):
            self.base_network.load_state_dict(state_dict['base_network'])
            self.class_network.load_state_dict(state_dict['class_network'])
            self.domain_network.load_state_dict(state_dict['domain_network'])

        def inference(self, output):
            predict_prob = torch.sigmoid(output)
            return predict_prob.cpu().numpy()

        def _train(self, loader):
            """Train the model for one epoch"""

            self.base_network.train()
            self.class_network.train()
            self.domain_network.train()

            train_class_loss = 0
            train_domain_loss = 0
            total = 0
            class_correct = 0
            domain_correct = 0
            for i, (images, expression, gender) in enumerate(loader):
                images, expression, gender = images.to(self.device), expression.to(self.device), gender.to(self.device)

                self.base_optimizer.zero_grad()
                self.class_optimizer.zero_grad()
                self.domain_optimizer.zero_grad()

                features = self.base_network(images)
                class_outputs = torch.squeeze(self.class_network(features))
                domain_outputs = self.domain_network(features)


                class_loss = self._criterion(class_outputs, expression)
                domain_loss = self._domain_criterion(domain_outputs, gender)

                total += expression.size(0)

                class_predicted = torch.where(torch.sigmoid(class_outputs) >= 0.5, torch.ones_like(class_outputs), torch.zeros_like(class_outputs)).long()
                #_, class_predicted = class_outputs.max(1)
                class_correct += class_predicted.eq(expression.long()).sum().item()
                _, domain_predicted = domain_outputs.max(1)
                domain_correct += domain_predicted.eq(gender.long()).sum().item()


                if i == 0 and self.epoch <= 10 and self.epoch % 3 == 0:
                    print()
                    print("class outputs:")
                    print(class_outputs)
                    print("class predicted")
                    print(class_predicted)
                    print("class loss")
                    print(class_loss)
                    print()
                    print("domain outputs:")
                    print(domain_outputs)
                    print("domain predicted")
                    print(domain_predicted)
                    print("domain loss")
                    print(domain_loss)


                # Update the main network
                if self.epoch % self.training_ratio == 0:
                    log_softmax = F.log_softmax(domain_outputs, dim=1)
                    confusion_loss = -log_softmax.mean(dim=1).mean()
                    loss = class_loss + self.alpha*confusion_loss
                    loss.backward()
                    self.class_optimizer.step()
                    self.base_optimizer.step()
                else:
                    # Update the domain classifier
                    domain_loss.backward()
                    self.domain_optimizer.step()

                train_class_loss += class_loss.item()
                train_domain_loss += domain_loss.item()
                #print('Train iteration', 
                #                {'class_loss': class_loss.item(),
                #                 'domain_loss': domain_loss.item(),
                #                 'domain_accuracy': 100.*correct/total},
                #                len(loader)*self.epoch + i)

                self.print_freq = 50

                #if self.print_freq and (i % self.print_freq == 0):
            print('Training epoch {}: [{}|{}], class loss:{}, class accuracy: {}, domain loss: {}, domain accuracy: {}'
                          .format(self.epoch, i+1, len(loader), 
                                  class_loss.item(), 100.*class_correct/total, domain_loss.item(),
                                  100.*domain_correct/total))

            #self.log_result('Train epoch', 
            #                {'class_loss': train_class_loss/len(loader),
            #                 'domain_loss': train_domain_loss/len(loader),
            #                 'domain_accuracy': 100.*correct/total}, 
            #                self.epoch)
            self.epoch += 1

        def _test(self, loader):
            """Compute model output on test set"""

            self.base_network.eval()
            self.class_network.eval()
            self.domain_network.eval()

            test_class_loss = 0
            test_domain_loss = 0
            total = 0
            class_correct = 0
            domain_correct = 0
            feature_list = []
            class_output_list = []
            domain_output_list = []
            with torch.no_grad():
                for i, (images, expression, gender) in enumerate(loader):
                    images, expression, gender = images.to(self.device), expression.to(self.device), gender.to(self.device)
                    features = self.base_network(images)
                    class_outputs = torch.squeeze(self.class_network(features))
                    domain_outputs = self.domain_network(features)

                    class_loss = self._criterion(class_outputs, expression)
                    domain_loss = self._domain_criterion(domain_outputs, gender)
                    test_class_loss += class_loss.item()
                    test_domain_loss += domain_loss.item()

                    total += expression.size(0)

                    class_predicted = torch.where(torch.sigmoid(class_outputs) >= 0.5, torch.ones_like(class_outputs), torch.zeros_like(class_outputs)).long()
                    class_correct += class_predicted.eq(expression.long()).sum().item()

                    _, domain_predicted = domain_outputs.max(1)
                    domain_correct += domain_predicted.eq(gender.long()).sum().item()

                    class_output_list.append(class_outputs)
                    domain_output_list.append(domain_outputs)
                    feature_list.append(features)

                #print('Testing epoch, class loss:{}, class accuracy: {}, domain loss: {}, domain accuracy: {}'
                #          .format(test_class_loss, 100.*class_correct/total, test_domain_loss, 
                #                  100.*domain_correct/total))
                return test_class_loss, test_domain_loss, torch.cat(class_output_list), 100.*class_correct/total, \
                       torch.cat(domain_output_list), torch.cat(feature_list), 100.*domain_correct/total

        def public_test(self, images):
            """Compute model output on test set"""

            self.base_network.eval()
            self.class_network.eval()
            self.domain_network.eval()


            features = self.base_network(images)
            class_outputs = torch.squeeze(self.class_network(features))
            domain_outputs = self.domain_network(features)

            return torch.sigmoid(class_outputs), domain_outputs

        def train(self):
            """Train the model for one epoch, evaluate on validation set and 
            save the best model
            """

            start_time = datetime.now()
            learning_rates1 = [1e-4, 1e-5, 1e-6] # class_optimizer
            learning_rates2 = [1e-3, 1e-4, 1e-5] # domain_optimizer
            epochs = [3, 6, 12]

            for learning_rate1, learning_rate2, epoch in zip(learning_rates1, learning_rates2, epochs):
                print()
                print('learning rate:', learning_rate1)
                print('epoch number:', epoch)
                self.base_optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.base_network.parameters()), 
                                      lr=learning_rate1)
                self.class_optimizer = torch.optim.Adam( 
                                          params=filter(lambda p: p.requires_grad, self.class_network.parameters()), 
                                          lr=learning_rate1, weight_decay=1e-5)
                self.domain_optimizer = torch.optim.Adam( 
                                          params=filter(lambda p: p.requires_grad, self.domain_network.parameters()), 
                                          lr=learning_rate2, weight_decay=1e-5)
                for i in range(epoch):
                    print()
                    self._train(self.train_loader)

                    dev_class_loss, dev_domain_loss, dev_class_output,dev_class_accuarcy, _,_, dev_domain_accuarcy = self._test(self.dev_loader)
                    dev_predict_prob = self.inference(dev_class_output)
                    dev_per_class_AP = compute_weighted_AP(self.dev_target, dev_predict_prob, 
                                                                 self.dev_class_weight)
                    dev_mAP = compute_mAP(dev_per_class_AP)


                    print('Testing epoch, class loss:{},  class accuracy: {}, mAP: {}, domain loss: {}, domain accuracy: {}'
                              .format(dev_class_loss/len(self.dev_loader), dev_class_accuarcy, dev_mAP,  
                                      dev_domain_loss/len(self.dev_loader), dev_domain_accuarcy))

                    #self.log_result('Dev epoch', 
                    #                {'class_loss': dev_class_loss/len(self.dev_loader), 
                    #                 'domain_loss': dev_domain_loss/len(self.dev_loader),
                    #                 'mAP': dev_mAP,
                    #                 'domain_accuracy': dev_domain_accuarcy},
                    #                self.epoch)
                    #if (self.epoch) > 1 and (dev_mAP > self.best_dev_mAP):
                    #    self.best_dev_mAP = dev_mAP
                    #
                    #    best_model_wts = copy.deepcopy(self.state_dict())
                    #    
                    #    
                    #    save_state_dict(self.state_dict(), os.path.join("./test/", 'best.pth'))
                    #if
                    #    print('best mAP, loss update:', dev_class_accuarcy, self.best_dev_mAP)
                    #else:
                    #    print('best mAP, loss not update, still:', dev_class_accuarcy, self.best_dev_mAP)

                #self.load_state_dict(best_model_wts)
                save_state_dict(self.state_dict(), os.path.join("./", 'uniconf_' + str(k) + '.pth'))
            duration = datetime.now() - start_time
            print('Finish training epoch {}, dev class loss: {}, dev doamin loss: {}, dev mAP: {},'\
                  'domain_accuracy: {}, time used: {}'
                  .format(self.epoch, dev_class_loss/len(self.dev_loader), 
                          dev_domain_loss/len(self.dev_loader), dev_mAP, dev_domain_accuarcy,
                          duration))

        def test(self):
            # Test and save the result
            state_dict = torch.load(os.path.join("./", 'uniconf_' + str(k) + '.pth'))
            self.load_state_dict(state_dict)



            test_class_loss, test_domain_loss, test_class_output, test_class_accuarcy, test_domain_output, \
                test_feature, test_domain_accuracy = self._test(self.test_loader)
            test_predict_prob = self.inference(test_class_output)
            test_per_class_AP = compute_weighted_AP(self.test_target, test_predict_prob, 
                                                         self.test_class_weight)
            test_mAP = compute_mAP(test_per_class_AP)
            print('per_class_AP', test_per_class_AP)
            print('mAP', test_mAP)
            print('domain_accuracy', test_domain_accuracy)
            return test_class_output.cpu().numpy(), test_domain_output.cpu().numpy(), test_feature.cpu().numpy()

    model = CelebaUniConfAdv()
    model.train()



    # Evaluate on Test Set

    # predict on test


    #model = CelebaUniConfAdv()
    #model.load_state_dict(torch.load('./test/best_uniconf.pth'))

    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    face_names = []
    scores = []
    preds = []

    true_labels = []

    gender_preds = []
    true_gender_labels = []

    for index, row in test_frame.iterrows():

        if index % 200 == 0:
            print(index)

        image_name = row['ImageName']

        image = dlib.load_rgb_image(image_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)
        image = image.to(device)

        outputs, gender_outputs = model.public_test(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        score = outputs * 1
        #score = np.exp(outputs) / np.sum(np.exp(outputs))
        pred = (score>=0.5)*1
        #pred = np.argmax(score)

        gender_outputs = gender_outputs.cpu().detach().numpy()
        gender_outputs = np.squeeze(gender_outputs)

        gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
        gender_pred = np.argmax(gender_score)

        face_names.append(image_name)
        scores.append(score)
        preds.append(pred)
        gender_preds.append(gender_pred)

        true_labels.append(row['Expression'])
        true_gender_labels.append(row['gender_preds'])

    test_result = pd.DataFrame(list(zip(face_names, scores, preds, true_labels, gender_preds, true_gender_labels)), 
                 columns = ['ImageName', 'ExpressionScore', 'Prediction', 'Expression', 'GenderPrediction', 'Gender'])
    test_result.head(10)

    # Accuracy
    test_result['CorrectOrNot'] = (test_result.Prediction == test_result.Expression)
    test_result['CorrectOrNot_gender'] = (test_result.GenderPrediction == test_result.Gender)


    dict_row = {}
    dict_row['model_random_state'] = k
    dict_row['test_accuracy'] = test_result.CorrectOrNot.mean()
    dict_row['test_gender_accuracy'] = test_result.CorrectOrNot_gender.mean()
    dict_row['test_male_true_proportion'] = (test_result[test_result.Gender == 1].Expression==1).mean()
    dict_row['test_female_true_proportion'] = (test_result[test_result.Gender == 0].Expression==1).mean()
    dict_row['test_male_predicted_proportion'] = (test_result[test_result.Gender == 1].Prediction==1).mean()
    dict_row['test_female_predicted_proportion'] = (test_result[test_result.Gender == 0].Prediction==1).mean()
    dict_row['test_male_average_score'] = test_result[test_result.Gender == 1].ExpressionScore.mean()
    dict_row['test_female_average_score'] = test_result[test_result.Gender == 0].ExpressionScore.mean()




    # Apply on Chicago Face

    chicago_df = pd.read_csv('ChicagoFace_selected_evaluation_set2.csv').iloc[:,0:4]
    chicago_df.head()

    def expression_string_to_num(s):
        if s == "Happy":
            return 3
        elif s == "Angry":
            return 0
        elif s == "Fear":
            return 2
        else:
            return 6

    chicago_df["Expression_num"] = chicago_df.Expression.apply(expression_string_to_num)


    chicago_df["happy"] = chicago_df.Expression_num.apply(lambda x: 1 if x == 3 else 0)




    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    face_names = []
    scores = []
    preds = []

    true_labels = []

    gender_preds = []

    for index, row in chicago_df.iterrows():

        #if index >= 100: break

        if index % 100 == 0:
            print(index)

        image_name = row['ImageName']


        image = dlib.load_rgb_image(image_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)
        image = image.to(device)

        outputs, gender_outputs = model.public_test(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        score = outputs * 1
        #score = np.exp(outputs) / np.sum(np.exp(outputs))
        pred = (score>=0.5)*1
        #pred = np.argmax(score)

        gender_outputs = gender_outputs.cpu().detach().numpy()
        gender_outputs = np.squeeze(gender_outputs)

        gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
        gender_pred = np.argmax(gender_score)

        face_names.append(image_name)
        scores.append(score)
        preds.append(pred)
        gender_preds.append(gender_pred)

        true_labels.append(row['happy'])

    chicago_result = pd.DataFrame([face_names, preds, scores, gender_preds]).T

    chicago_result.columns = ['ImageName', 'expression_preds', 'expression_scores', 'gender_preds_model']


    chicago_result["happiness_score"] = chicago_result.expression_scores.apply(lambda x: x)


    num_happy = (chicago_df.happy == 1).sum()
    threshold = chicago_result.sort_values(by = ['happiness_score'], ascending = False).reset_index(drop = True).iloc[num_happy]['happiness_score']

    chicago_result['expression_preds_relabeled'] = chicago_result.happiness_score.apply(lambda x : 1 if x > threshold else 0)


    chicago_df_merged = pd.merge(chicago_df, chicago_result, on = ['ImageName'], how = 'left')

    chicago_df_merged["true_gender"] = chicago_df_merged.Gender.apply(lambda x: 1 if x == "M" else 0)


    chicago_df_merged['CorrectOrNot'] = (chicago_df_merged.expression_preds_relabeled == chicago_df_merged.happy)
    dict_row['selected_2_chicago_accuracy'] = chicago_df_merged.CorrectOrNot.mean() 

    dict_row['selected_2_chicago_gender_accuracy'] = (chicago_df_merged.gender_preds_model == chicago_df_merged.true_gender).mean()

    chicago_male_frame = chicago_df_merged.loc[chicago_df_merged['Gender'] == 'M']
    chicago_female_frame = chicago_df_merged.loc[chicago_df_merged['Gender'] == 'F']

    # Accuracy between males and females
    dict_row['selected_2_chicago_accuracy_male'] = (chicago_male_frame.expression_preds_relabeled == chicago_male_frame.happy).mean()
    dict_row['selected_2_chicago_accuracy_female'] = (chicago_female_frame.expression_preds_relabeled == chicago_female_frame.happy).mean()

    # True proportion
    dict_row['selected_2_chicago_true_proportion'] = chicago_male_frame.happy.mean()

    # Prediction proportion
    dict_row['selected_2_chicago_male_predicted_proportion'] = chicago_male_frame.expression_preds_relabeled.mean()
    dict_row['selected_2_chicago_female_predicted_proportion'] = chicago_female_frame.expression_preds_relabeled.mean()
    dict_row['selected_2_chicago_bias'] = chicago_female_frame.expression_preds_relabeled.mean() - chicago_male_frame.expression_preds_relabeled.mean()

    # Prediction proportion
    dict_row['selected_2_chicago_male_predicted_proportion_raw'] = chicago_male_frame.expression_preds.mean()
    dict_row['selected_2_chicago_female_predicted_proportion_raw'] = chicago_female_frame.expression_preds.mean()
    dict_row['selected_2_chicago_bias_raw'] = chicago_female_frame.expression_preds.mean() - chicago_male_frame.expression_preds.mean()

    # Average Happiness Score
    dict_row['selected_2_chicago_male_score'] = chicago_male_frame.happiness_score.mean()
    dict_row['selected_2_chicago_female_score'] = chicago_female_frame.happiness_score.mean()

    # Average Happiness Score among neutral faces
    dict_row['selected_2_chicago_male_score_neutral_faces'] = chicago_male_frame[chicago_male_frame.Expression_num == 6].happiness_score.mean()
    dict_row['selected_2_chicago_female_score_neutral_faces'] = chicago_female_frame[chicago_female_frame.Expression_num == 6].happiness_score.mean()


    result_rows_list.append(dict_row)
    results_df = pd.DataFrame(result_rows_list) 
    print(results_df)
    results_df.to_csv('model_evaluation_result.csv', index = False)

    del model
    torch.cuda.empty_cache()

16000 train faces, 2000 validation faces, 2000 test faces

learning rate: 0.0001
epoch number: 3


class outputs:
tensor([ 0.1301,  0.2020,  0.1576,  0.1901, -0.0138,  0.1094,  0.0653,  0.0641,
         0.2365,  0.1680,  0.0842,  0.0122, -0.0315, -0.3327,  0.1224,  0.0914,
         0.1671,  0.0684, -0.1949,  0.2487, -0.2491, -0.0163, -0.1296,  0.0029,
         0.1143,  0.0228, -0.0279, -0.0117, -0.0742, -0.1040,  0.2357,  0.0792,
         0.0379, -0.0923,  0.1328, -0.1630, -0.0473, -0.1158, -0.0934,  0.0904,
         0.0426,  0.1829,  0.0104,  0.0923, -0.0927,  0.0939, -0.4418, -0.0518,
        -0.1590,  0.1096, -0.0011, -0.1568,  0.2121,  0.0016, -0.1645, -0.1048,
        -0.1314,  0.0602, -0.2841, -0.0418, -0.1270, -0.2845, -0.1357, -0.1178],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
class predicted
tensor([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 1, 0, 

Training epoch 4: [250|250], class loss:0.36340218782424927, class accuracy: 93.30625, domain loss: 0.7218430638313293, domain accuracy: 55.10625
Testing epoch, class loss:0.38103272998705506,  class accuracy: 86.15, mAP: 0.8683019766944686, domain loss: 0.6827314589172602, domain accuracy: 60.1

Training epoch 5: [250|250], class loss:0.3589991331100464, class accuracy: 93.44375, domain loss: 0.7132853865623474, domain accuracy: 56.33125
Testing epoch, class loss:0.38103272998705506,  class accuracy: 86.15, mAP: 0.8683019766944686, domain loss: 0.6815697830170393, domain accuracy: 60.6


class outputs:
tensor([-4.8336,  0.3287, -3.6064, -4.6183, -2.0164, -0.2828, -5.6905, -4.3712,
        -5.2223, -4.7854,  2.2554, -4.5005, -6.8854, -3.8829,  4.0559, -0.9471,
        -1.9835,  2.7917, -5.7913,  1.7951, -2.3411, -5.5428,  1.4570,  4.3726,
        -5.7664, -5.5461,  6.8044,  5.2518, -2.0765, -4.3465, -4.3430,  5.0229,
        -3.4936,  0.3784,  0.4420, -3.7957, -5.5313,  0.0115, -3.5880

Training epoch 10: [250|250], class loss:0.1983119249343872, class accuracy: 97.58125, domain loss: 0.7167351245880127, domain accuracy: 55.5
Testing epoch, class loss:0.4365308010019362,  class accuracy: 86.15, mAP: 0.8582275482102039, domain loss: 0.6841459814459085, domain accuracy: 61.05

Training epoch 11: [250|250], class loss:0.20091408491134644, class accuracy: 97.59375, domain loss: 0.69389808177948, domain accuracy: 57.03125
Testing epoch, class loss:0.4365308010019362,  class accuracy: 86.15, mAP: 0.8582275482102039, domain loss: 0.6837697010487318, domain accuracy: 60.65

Training epoch 12: [250|250], class loss:0.17425507307052612, class accuracy: 97.6625, domain loss: 0.687990128993988, domain accuracy: 52.575
Testing epoch, class loss:0.44140707328915596,  class accuracy: 86.1, mAP: 0.8556180017080395, domain loss: 0.6884503811597824, domain accuracy: 61.9

Training epoch 13: [250|250], class loss:0.16574952006340027, class accuracy: 98.025, domain loss: 0.69718295335769

Training epoch 0: [250|250], class loss:0.3580441474914551, class accuracy: 81.275, domain loss: 0.6952103972434998, domain accuracy: 50.25
Testing epoch, class loss:0.32580867735669017,  class accuracy: 86.45, mAP: 0.8818379792000343, domain loss: 0.6930690445005894, domain accuracy: 40.4

Training epoch 1: [250|250], class loss:0.3249955475330353, class accuracy: 88.85625, domain loss: 0.6754697561264038, domain accuracy: 56.3875
Testing epoch, class loss:0.32648892886936665,  class accuracy: 86.25, mAP: 0.8812850280124198, domain loss: 0.6781913638114929, domain accuracy: 56.1

Training epoch 2: [250|250], class loss:0.33062225580215454, class accuracy: 88.8875, domain loss: 0.7150474190711975, domain accuracy: 56.825
Testing epoch, class loss:0.32648892886936665,  class accuracy: 86.25, mAP: 0.8812850280124198, domain loss: 0.6745137479156256, domain accuracy: 61.1

learning rate: 1e-05
epoch number: 6


class outputs:
tensor([ 3.5302, -3.5707, -0.8580, -0.1885, -3.1307, -3.1159, -

Training epoch 6: [250|250], class loss:0.12661783397197723, class accuracy: 94.275, domain loss: 0.7055932879447937, domain accuracy: 50.96875
Testing epoch, class loss:0.4495045836083591,  class accuracy: 86.45, mAP: 0.8663786060253399, domain loss: 0.6897610686719418, domain accuracy: 53.25

Training epoch 7: [250|250], class loss:0.10431375354528427, class accuracy: 97.25, domain loss: 0.6788730025291443, domain accuracy: 55.71875
Testing epoch, class loss:0.4480992378666997,  class accuracy: 86.35, mAP: 0.866087372927852, domain loss: 0.6807694807648659, domain accuracy: 61.3

Training epoch 8: [250|250], class loss:0.10846435278654099, class accuracy: 97.25, domain loss: 0.6770976781845093, domain accuracy: 56.54375
Testing epoch, class loss:0.4480992378666997,  class accuracy: 86.35, mAP: 0.866087372927852, domain loss: 0.6790521275252104, domain accuracy: 61.55

learning rate: 1e-06
epoch number: 12


class outputs:
tensor([ 3.8719, -4.9440,  1.9689,  4.4515, -4.6855, -5.0787, 

...

Training epoch 0: [250|250], class loss:0.39034467935562134, class accuracy: 81.34375, domain loss: 0.7019896507263184, domain accuracy: 50.5625
Testing epoch, class loss:0.33968701399862766,  class accuracy: 85.95, mAP: 0.8636525291880549, domain loss: 0.6961531732231379, domain accuracy: 39.4

Training epoch 1: [250|250], class loss:0.3693617284297943, class accuracy: 87.80625, domain loss: 0.7104107141494751, domain accuracy: 57.6875
Testing epoch, class loss:0.3416293174959719,  class accuracy: 85.55, mAP: 0.8643296372057307, domain loss: 0.6752764228731394, domain accuracy: 52.15

Training epoch 2: [250|250], class loss:0.3685336709022522, class accuracy: 87.76875, domain loss: 0.7227420806884766, domain accuracy: 57.975
Testing epoch, class loss:0.3416293174959719,  class accuracy: 85.55, mAP: 0.8643296372057307, domain loss: 0.6741720736026764, domain accuracy: 47.4

learning rate: 1e-05
epoch number: 6


class outputs:
tensor([-1.6763,  6.0353, -1.6094, -2.2617, -1.8309,  1.795

Training epoch 6: [250|250], class loss:0.2508198618888855, class accuracy: 94.15, domain loss: 0.6863307952880859, domain accuracy: 51.23125
Testing epoch, class loss:0.4232567804865539,  class accuracy: 85.6, mAP: 0.8501556567849217, domain loss: 0.6903848722577095, domain accuracy: 54.55

Training epoch 7: [250|250], class loss:0.2252047061920166, class accuracy: 97.225, domain loss: 0.6601918935775757, domain accuracy: 55.825
Testing epoch, class loss:0.4235530784353614,  class accuracy: 85.75, mAP: 0.8497928052240807, domain loss: 0.6832386516034603, domain accuracy: 60.5

Training epoch 8: [250|250], class loss:0.22408923506736755, class accuracy: 97.26875, domain loss: 0.681687593460083, domain accuracy: 55.85625
Testing epoch, class loss:0.4235530784353614,  class accuracy: 85.75, mAP: 0.8497928052240807, domain loss: 0.6816285941749811, domain accuracy: 60.85

learning rate: 1e-06
epoch number: 12


class outputs:
tensor([-3.6639,  6.8493, -6.2267, -5.4227, -3.0034,  4.3067, -

Finish training epoch 21, dev class loss: 0.45972800021991134, dev doamin loss: 0.6825358085334301, dev mAP: 0.8387860930331732,domain_accuracy: 60.95, time used: 0:51:28.317845
0
200
400
600
800
1000
1200
1400
1600
1800
0
100
200
300
400
500
600
700
800
900
1000
1100
0
100
200
300
400
500
0
100
200
300
400
500
600
700
   model_random_state  test_accuracy  test_gender_accuracy  \
0                   1         0.8450                0.5985   
1                   2         0.8565                0.6160   
2                   3         0.8630                0.6045   
3                   4         0.8575                0.6100   

   test_male_true_proportion  test_female_true_proportion  \
0                   0.302895                     0.398162   
1                   0.319911                     0.409712   
2                   0.295570                     0.356340   
3                   0.288262                     0.397554   

   test_male_predicted_proportion  test_female_predicted_propo

Testing epoch, class loss:0.3373802420683205,  class accuracy: 86.4, mAP: 0.8662239720908407, domain loss: 0.6933228150010109, domain accuracy: 65.45

Training epoch 2: [250|250], class loss:0.29517242312431335, class accuracy: 89.6625, domain loss: 0.7386156320571899, domain accuracy: 58.06875
Testing epoch, class loss:0.3373802420683205,  class accuracy: 86.4, mAP: 0.8662239720908407, domain loss: 0.6905671637505293, domain accuracy: 66.35

learning rate: 1e-05
epoch number: 6


class outputs:
tensor([ 0.3555, -3.4150, -0.6483, -3.1381, -2.9582, -3.2947, -1.2859, -0.7418,
        -3.6357, -1.4021,  2.8323,  2.8841, -2.0362, -1.9189, -0.2052, -0.7702,
        -1.7162, -0.7009, -1.0768, -1.7658,  1.6557, -0.9076, -0.0438,  0.1942,
         3.5960, -0.7873,  3.9240, -1.3705, -2.3158,  5.3328, -3.7716, -2.5562,
        -4.9491, -2.2194,  3.4727, -3.8697, -0.5664,  4.0815, -3.6598,  2.9857,
        -1.5657, -1.1151, -2.1858, -4.0707, -4.6547, -0.1145, -1.1701, -0.5500,
         3.9589, -2

Testing epoch, class loss:0.4314512889832258,  class accuracy: 85.85, mAP: 0.8505639861440855, domain loss: 0.6849650628864765, domain accuracy: 60.15

Training epoch 8: [250|250], class loss:0.14186108112335205, class accuracy: 97.5875, domain loss: 0.7089899778366089, domain accuracy: 55.98125
Testing epoch, class loss:0.4314512889832258,  class accuracy: 85.85, mAP: 0.8505639861440855, domain loss: 0.684381976723671, domain accuracy: 60.55

learning rate: 1e-06
epoch number: 12


class outputs:
tensor([ 2.3397, -5.0255, -5.3930, -4.9522, -5.5832, -5.4035, -2.9333,  3.7925,
        -6.0555, -1.1613,  4.5215,  4.1061, -4.6925, -2.8906, -2.5909, -3.9152,
        -6.0024,  3.4787, -4.2688, -4.0133,  4.9800, -3.5128,  3.6225,  4.8444,
         3.5241, -5.2821,  5.3360, -2.9603, -3.8899,  6.7302, -5.6956,  1.0060,
        -8.3084, -4.0222,  3.6554, -5.9203,  1.4218,  4.7647, -5.2669,  3.6459,
        -8.4148, -4.9724, -2.5287, -7.1084, -6.1893, -5.0786, -3.9623, -0.2826,
         5.7430, 